In [7]:
import numpy as np
import pandas as pd
from sklearn import preprocessing
import lightgbm as lgb
import h2o
from h2o.automl import H2OAutoML
from sklearn.model_selection import train_test_split
#import pandas_profiling

In [73]:
train_df = pd.read_csv('../data/train.csv')
test_df = pd.read_csv('../data/test.csv')

In [8]:
from sklearn.model_selection import StratifiedKFold
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score

n_folds = 0
random_seed = 26
model = 'h2o'


model_name = "{0}_{1}_folds".format(model, n_folds)
print("Model: {}".format(model_name))

Model: h2o_0_folds


In [13]:
# X = train_df.iloc[:,2:]
# y = train_df['target']

train, valid = train_test_split(train_df, test_size=0.2)

In [14]:
train.head(2)

,ID_code,target,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
140686,train_140686,0,13.8544,-1.7014,11.1719,8.2697,9.7087,6.4796,7.4362,20.6544,...,7.4810,5.4282,-0.1123,-1.4447,14.8711,-2.0421,-7.2447,8.6219,16.4862,-1.7213
139605,train_139605,0,9.5581,-3.8057,8.2713,6.4919,9.8153,-1.1829,3.9474,20.1171,...,1.8109,11.3583,3.3764,4.1503,16.7433,-0.6018,7.0178,8.5348,18.7204,-0.4525


In [22]:
h2o.init()

# Convert to an h2o frame
df_train_h2o = h2o.H2OFrame(train_df)
df_test_h2o = h2o.H2OFrame(test_df)

feats = [i for i in train.columns if i not in ['target','ID_code']]
# For binary classification, response should be a factor
df_train_h2o['target'] = df_train_h2o['target'].asfactor()
#df_valid_h2o['target'] = df_valid_h2o['target'].asfactor()



Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,3 mins 35 secs
H2O cluster timezone:,Australia/Hobart
H2O data parsing timezone:,UTC
H2O cluster version:,3.22.1.4
H2O cluster version age:,10 days
H2O cluster name:,H2O_from_python_TanQ_xfmqls
H2O cluster total nodes:,1
H2O cluster free memory:,7.372 Gb
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster status:,"locked, healthy"


Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


H2OResponseError: Server error java.lang.IllegalArgumentException:
  Error: Field = stopping_metric cannot be set to value = auc
  Request: POST /99/AutoMLBuilder
    json: {'input_spec': {'response_column': 'target', 'training_frame': 'py_1_sid_88cf', 'sort_metric': 'auc', 'ignored_columns': ['ID_code']}, 'build_control': {'stopping_criteria': {'max_runtime_secs': 300, 'max_models': 100, 'stopping_metric': 'auc', 'stopping_rounds': 3, 'seed': 183}, 'nfolds': 5, 'keep_cross_validation_predictions': False, 'keep_cross_validation_models': False, 'keep_cross_validation_fold_assignment': False, 'project_name': 'automl_py_1_sid_88cf'}, 'build_models': {'exclude_algos': None}}


In [24]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml_atp = h2o.automl.H2OAutoML(
#                            max_runtime_secs=300,
#                            max_models=100,
                           stopping_metric='AUC',
                           sort_metric='AUC',
                           balance_classes=False,
                           seed=183
                          )

aml_atp.train(x=feats, y='target', training_frame=df_train_h2o)

# View the AutoML Leaderboard
lb = aml_atp.leaderboard
lb.head()  

AutoML progress: |████████████████████████████████████████████████████████| 100%


model_id,auc,logloss,mean_per_class_error,rmse,mse
StackedEnsemble_AllModels_AutoML_20190226_122855,0.885387,0.218676,0.246727,0.247943,0.0614756
StackedEnsemble_BestOfFamily_AutoML_20190226_122855,0.881965,0.220265,0.252012,0.249084,0.0620427
GBM_2_AutoML_20190226_122855,0.872152,0.228938,0.26975,0.256771,0.0659312
GBM_1_AutoML_20190226_122855,0.87131,0.23075,0.271458,0.257624,0.0663699
GBM_3_AutoML_20190226_122855,0.869851,0.230723,0.261024,0.258014,0.0665711
GLM_grid_1_AutoML_20190226_122855_model_1,0.859708,0.232019,0.26577,0.257973,0.0665502
XRT_1_AutoML_20190226_122855,0.804783,0.279096,0.301113,0.282568,0.0798448
DRF_1_AutoML_20190226_122855,0.803282,0.27881,0.307709,0.282432,0.0797678


In [69]:
test_preds = aml_atp.predict(df_test_h2o)['p1'].as_data_frame()

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [70]:
test_preds.shape

(200000, 1)

In [71]:
test_df.shape

(200000, 201)

In [74]:
test_ids = test_df.ID_code.values
print("Saving submission file")
sample = pd.read_csv('../data/sample_submission.csv')
sample.target = test_preds.astype(float)
sample.ID_code = test_ids
sample.to_csv('../submissions/submission_{}_{}.csv'.format(model_name,str(0.885387)), index=False)





Saving submission file


In [77]:
sample.target.mean()

0.09229926349031733

In [43]:
df_test_h2o = h2o.H2OFrame(test_df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [44]:
df_test_h2o.shape

(200001, 201)

In [42]:
test_df.shape

(200000, 201)

In [48]:
df_test_h2o.tail()

ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,var_9,var_10,var_11,var_12,var_13,var_14,var_15,var_16,var_17,var_18,var_19,var_20,var_21,var_22,var_23,var_24,var_25,var_26,var_27,var_28,var_29,var_30,var_31,var_32,var_33,var_34,var_35,var_36,var_37,var_38,var_39,var_40,var_41,var_42,var_43,var_44,var_45,var_46,var_47,var_48,var_49,var_50,var_51,var_52,var_53,var_54,var_55,var_56,var_57,var_58,var_59,var_60,var_61,var_62,var_63,var_64,var_65,var_66,var_67,var_68,var_69,var_70,var_71,var_72,var_73,var_74,var_75,var_76,var_77,var_78,var_79,var_80,var_81,var_82,var_83,var_84,var_85,var_86,var_87,var_88,var_89,var_90,var_91,var_92,var_93,var_94,var_95,var_96,var_97,var_98,var_99,var_100,var_101,var_102,var_103,var_104,var_105,var_106,var_107,var_108,var_109,var_110,var_111,var_112,var_113,var_114,var_115,var_116,var_117,var_118,var_119,var_120,var_121,var_122,var_123,var_124,var_125,var_126,var_127,var_128,var_129,var_130,var_131,var_132,var_133,var_134,var_135,var_136,var_137,var_138,var_139,var_140,var_141,var_142,var_143,var_144,var_145,var_146,var_147,var_148,var_149,var_150,var_151,var_152,var_153,var_154,var_155,var_156,var_157,var_158,var_159,var_160,var_161,var_162,var_163,var_164,var_165,var_166,var_167,var_168,var_169,var_170,var_171,var_172,var_173,var_174,var_175,var_176,var_177,var_178,var_179,var_180,var_181,var_182,var_183,var_184,var_185,var_186,var_187,var_188,var_189,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198
test_199990,5.5416,1.734,9.6938,5.0126,11.3049,-15.9906,5.0937,17.796,-3.105,6.9197,8.2548,-2.8756,14.1546,14.7619,11.4564,15.0365,13.8591,-6.5502,20.9181,5.0439,15.0863,30.4997,7.7843,3.1669,11.9212,14.0191,-1.7656,0.7349,5.2116,8.6242,-21.4105,10.7667,-4.493,15.6523,12.3655,0.7796,3.5171,4.2829,15.8226,-1.6903,0.1326,14.2905,10.8666,10.8967,11.1282,-15.3843,11.2523,-0.7021,40.3556,13.8107,12.1475,25.1136,-6.0759,6.6962,-4.4275,5.9248,12.4274,5.3593,-1.0296,8.8086,16.5595,-24.3924,-0.1339,9.8999,6.2393,8.0134,6.2466,-5.49,5.016,-2.2404,6.5521,0.3543,-2.4493,28.4266,2.9535,7.262,12.5272,16.144,6.8229,16.0341,15.7816,12.7976,1.1177,10.4361,-0.0018,12.521,6.95,23.6482,7.1241,5.1609,-21.9156,6.9884,9.8085,10.4827,11.1193,-0.5496,19.8106,29.2678,1.6507,-1.8869,-6.0659,17.8818,23.6784,1.5022,9.1935,4.5686,7.4722,22.1987,14.2272,23.883,8.7698,5.7006,2.3303,6.5322,3.1956,0.6683,3.7992,22.1745,1.8399,1.8231,14.4718,9.5019,-5.4746,13.0737,-1.2591,12.4395,12.2605,3.4311,-2.8784,15.5806,13.2464,0.8203,10.0867,7.0273,-2.9489,6.855,27.5006,6.4966,-4.945,7.9073,5.7229,2.329,11.9754,7.8323,9.5304,2.1782,10.8261,-1.2333,4.0181,-18.9335,16.3227,15.2164,9.6197,18.9541,0.3094,2.3146,12.8678,-10.2213,16.1873,14.2569,13.9966,5.4965,5.3224,4.8996,3.7209,14.2065,3.1629,-15.7246,4.3909,5.6583,-9.2583,7.9383,11.477,-9.1503,12.6929,8.7636,-3.8056,8.1242,12.447,0.495,-0.6494,7.734,13.9948,5.1769,-4.0009,-8.5752,11.4089,-11.8738,16.9443,0.5255,1.1663,5.4021,1.4266,0.7912,20.6181,0.5917,11.6931,9.6883,12.6723
test_199991,8.7935,-4.0646,9.948,8.6947,11.0497,-0.5129,5.641,21.5338,5.6578,5.3441,-7.2666,-4.5693,14.2109,3.9747,5.9191,14.2149,7.3641,-10.9504,15.1152,5.3521,17.5164,0.4992,6.2773,2.9737,12.7908,14.428,-4.8483,-1.4386,6.7189,5.8642,-12.2366,9.0299,3.894,15.2421,11.5398,3.6696,-0.2825,8.373,13.9044,0.2526,0.1411,15.5557,11.3582,11.5342,5.3382,-21.5913,16.1228,-11.2815,9.1485,27.2697,13.0205,12.1711,-8.6204,5.2903,-0.1652,20.8653,15.4474,7.1057,1.6742,8.5306,7.4351,1.4279,1.8538,4.4484,5.6925,-2.7996,4.0526,3.232,5.0262,0.0389,27.6938,1.0398,-1.1977,18.566,7.5126,8.3362,8.7411,20.466,3.9469,14.5636,17.2183,16.6168,2.7772,-2.7028,1.9433,23.6108,-1.2721,5.2517,9.113,6.6544,-1.1853,7.0726,11.1609,10.8538,13.0827,0.2404,15.6519,0.8124,1.8064,-3.0838,-4.1264,21.941,31.1745,1.616,12.8227,3.9001,13.3268,26.1281,14.2103,15.4257,9.9754,6.3793,1.527,2.1524,3.1832,0.0462,2.82,16.487,-9.4411,-2.5926,39.8529,9.6855,5.0041,20.0428,2.4075,12.4137,13.8758,-1.1914,5.0234,15.0224,11.4531,0.9663,8.6081,6.88

In [47]:
test_df.tail()

,ID_code,var_0,var_1,var_2,var_3,var_4,var_5,var_6,var_7,var_8,...,var_190,var_191,var_192,var_193,var_194,var_195,var_196,var_197,var_198,var_199
199995,test_199995,13.1678,1.0136,10.4333,6.7997,8.5974,-4.1641,4.8579,14.7625,-2.7239,...,2.0544,9.6849,4.6734,-1.3660,12.8721,1.2013,-4.6195,9.1568,18.2102,4.8801
199996,test_199996,9.7171,-9.1462,7.3443,9.1421,12.8936,3.0191,5.6888,18.8862,5.0915,...,5.0071,6.6548,1.8197,2.4104,18.9037,-0.9337,2.9995,9.1112,18.1740,-20.7689
199997,test_199997,11.6360,2.2769,11.2074,7.7649,12.6796,11.3224,5.3883,18.3794,1.6603,...,5.1536,2.6498,2.4937,-0.0637,20.0609,-1.1742,-4.1524,9.1933,11.7905,-22.2762
199998,test_199998,13.5745,-0.5134,13.6584,7.4855,11.2241,-11.3037,4.1959,16.8280,5.3208,...,3.4259,8.5012,2.2713,5.7621,17.0056,1.1763,-2.3761,8.1079,8.7735,-0.2122
199999,test_199999,10.4664,1.8070,10.2277,6.0654,10.0258,1.0789,4.8879,14.4892,-0.5902,...,0.1398,9.2828,1.3601,4.8985,20.0926,-1.3048,-2.5981,10.3378,14.3340,-7.7094


In [55]:
sample = pd.read_csv('../data/sample_submission.csv')
sample['target'] = test_preds.astype(float)

In [59]:
df_train_h2o.shape

(200000, 202)

In [61]:
df_test_h2o.shape

(200001, 201)

In [60]:
test_df.shape

(200000, 201)

In [62]:
df_test_h2o = h2o.H2OFrame(test_df)

Parse progress: |█████████████████████████████████████████████████████████| 100%


(200001, 201)

In [67]:
df_test_h2o = h2o.import_file('../data/test.csv')

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [68]:
df_test_h2o.shape

(200000, 201)